In [1]:
import numpy as np
import pandas as pd
import scipy.stats as sps
from statsmodels.stats.proportion import proportion_confint
#from tqdm.notebook import tqdm as tqdm_notebook
from tqdm import tqdm as tqdm_notebook
from collections import namedtuple
import seaborn as sns
import itertools
sns.set(font_scale=1.5, palette='Set2')
ExperimentComparisonResults = namedtuple('ExperimentComparisonResults',
                                        ['pvalue', 'effect', 'ci_length', 'left_bound', 'right_bound', 'var'])

In [2]:
#T-test: на вход принимает два массива для КГ и тестовой группы с метриками по пользователям
def absolute_ttest(control, test):
    mean_control = np.mean(control)
    mean_test = np.mean(test)
    var_mean_control  = np.var(control) / len(control)
    var_mean_test  = np.var(test) / len(test)

    var = (np.var(control) + np.var(test))/2.0

    difference_mean = mean_test - mean_control
    difference_mean_var = var_mean_control + var_mean_test
    difference_distribution = sps.norm(loc=difference_mean, scale=np.sqrt(difference_mean_var))

    left_bound, right_bound = difference_distribution.ppf([0.025, 0.975])
    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(difference_distribution.cdf(0), difference_distribution.sf(0))
    effect = difference_mean

    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound, var)

#T-test с CUPED преобразоваием: также добавляем массивы для ковариат по пользователям
def cuped_ttest(control, test, control_before, test_before):
    theta = (np.cov(control, control_before)[0, 1] + np.cov(test, test_before)[0, 1]) /\
                (np.var(control_before) + np.var(test_before))

    control_cup = control - theta * control_before
    test_cup = test - theta * test_before
    return absolute_ttest(control_cup, test_cup)

In [3]:
# Создаем выборки для КГ и тестовой группы
control_before = sps.expon(scale=1000).rvs(3880) + sps.norm(loc=0, scale=1000).rvs(3880)
control_before[control_before < 0] = 0
control = control_before + sps.norm(loc=0, scale=1000).rvs(3880)
control[control < 0] = 0

test_before = sps.expon(scale=1000).rvs(3880) + sps.norm(loc=0, scale=1000).rvs(3880)
test_before[test_before < 0] = 0
test = test_before + sps.norm(loc=0, scale=1000).rvs(3880)
test[test < 0] = 0
test *= 1.07 # Истинный рост в тестовой группе

In [4]:
np.corrcoef(control + test, control_before + test_before)[0,1]

0.7984324631679355

In [5]:
cuped_ttest(control, test, control_before, test_before)

ExperimentComparisonResults(pvalue=2.4655107340847505e-09, effect=116.12326891707107, ci_length=76.32726617335288, left_bound=77.95963583039463, right_bound=154.2869020037475, var=735537.7577146343)

**Проверка методики на симуляции тестов**

In [6]:
#T-test с некорректным CUPED - увеличивает ошибки первого рода. Часто данная метрика упоминается в статьях
def cuped_ttest_wrong(control, test, control_before, test_before):
    theta = (np.cov(control, control_before)[0, 1] + np.cov(test, test_before)[0, 1]) /\
                (np.var(control_before) + np.var(test_before))

    control_cup = control - theta * (control_before - np.mean(control_before))
    test_cup = test - theta * (test_before - np.mean(test_before))
    return absolute_ttest(control_cup, test_cup)

In [7]:
# Сохраняем данные экспериментов
bad_cnt_list_1 = []
bad_cnt_list_2 = []
bad_cnt_list_3 = []

effect_list_1 = []
effect_list_2 = []
effect_list_3 = []

ci_length_list_1 = []
ci_length_list_2 = []
ci_length_list_3 = []

var_list_1 = []
var_list_2 = []
var_list_3 = []

corr_coef_list = []

# Цикл проверки
N = 10000
for i in tqdm_notebook(range(N)):
    # Тестирую A/A-тест
    control_before = sps.expon(scale=1000).rvs(3880) + sps.norm(loc=0, scale=1000).rvs(3880)
    control_before[control_before < 0] = 0
    control = control_before + sps.norm(loc=0, scale=1500).rvs(3880)
    control[control < 0] = 0

    test_before = sps.expon(scale=1000).rvs(3880) + sps.norm(loc=0, scale=1000).rvs(3880)
    test_before[test_before < 0] = 0
    test = test_before + sps.norm(loc=0, scale=1500).rvs(3880)
    test[test < 0] = 0

    test *= 1.07 # Истинный рост в тестовой группе

    corr_coef_list.append(np.corrcoef(control + test, control_before + test_before)[0,1])

    # Запускаю критерий
    p_value_1, effect_1, ci_length_1, _, _, var_1 = cuped_ttest(control, test, control_before, test_before)
    p_value_2, effect_2, ci_length_2, _, _, var_2 = absolute_ttest(control, test)
    p_value_3, effect_3, ci_length_3, _, _, var_3 = cuped_ttest_wrong(control, test, control_before, test_before)

    # Смотрю результаты
    if p_value_1 < 0.05:
        bad_cnt_list_1.append(1)
    else: bad_cnt_list_1.append(0)

    if p_value_2 < 0.05:
        bad_cnt_list_2.append(1)
    else: bad_cnt_list_2.append(0)

    if p_value_3 < 0.05:
        bad_cnt_list_3.append(1)
    else: bad_cnt_list_3.append(0)

    effect_list_1.append(effect_1)
    effect_list_2.append(effect_2)
    effect_list_3.append(effect_3)

    ci_length_list_1.append(ci_length_1)
    ci_length_list_2.append(ci_length_2)
    ci_length_list_3.append(ci_length_3)

    var_list_1.append(var_1)
    var_list_2.append(var_2)
    var_list_3.append(var_3)

100%|██████████| 10000/10000 [02:39<00:00, 62.52it/s]


In [8]:
# Строю доверительный интервал для доли ошибок у критерия.
left_real_level, right_real_level = proportion_confint(count = sum(bad_cnt_list_2), nobs = N, alpha=0.05, method='wilson')
print(f"Доля отклонений нулевой гипотезы: {round(sum(bad_cnt_list_2) / N, 4)};"
      f" CI:[{round(left_real_level, 4)}, {round(right_real_level, 4)}]")

left_real_level, right_real_level = proportion_confint(count = sum(bad_cnt_list_1), nobs = N, alpha=0.05, method='wilson')
print(f"Доля отклонений нулевой гипотезы Cuped: {round(sum(bad_cnt_list_1) / N, 4)};"
      f" CI:[{round(left_real_level, 4)}, {round(right_real_level, 4)}]")
left_real_level, right_real_level = proportion_confint(count = sum(bad_cnt_list_3), nobs = N, alpha=0.05, method='wilson')

print(f"Доля отклонений нулевой гипотезы Cuped_wrong: {round(sum(bad_cnt_list_3) / N, 4)};"
      f" доверительный интервал: [{round(left_real_level, 4)}, {round(right_real_level, 4)}]\n")

print(f"Средняя длина доверительного интервала: {round(sum(ci_length_list_2) / N, 4)}")
print(f"Средняя длина доверительного интервала cuped: {round(sum(ci_length_list_1) / N, 4)}")
print(f"Средняя длина доверительного интервала Cuped_wrong: {round(sum(ci_length_list_3) / N, 4)}\n")

print(f"Средний эффект: {round(sum(effect_list_2) / N, 4)}")
print(f"Средний эффект Cuped: {round(sum(effect_list_1) / N, 4)}")
print(f"Средний эффект Cuped_wrong: {round(sum(effect_list_3) / N, 4)}\n")

print(f"Средняя дисперсия: {round(sum(var_list_2) / N, 4)}")
print(f"Средняя дисперсия Cuped: {round(sum(var_list_1) / N, 4)}")
print(f"Средняя дисперсия Cuped_wrong: {round(sum(var_list_3) / N, 4)}\n")


print(f"Корреляция: {round(sum(corr_coef_list) /N, 4)}")

Доля отклонений нулевой гипотезы: 0.798; CI:[0.79, 0.8058]
Доля отклонений нулевой гипотезы Cuped: 0.9598; CI:[0.9558, 0.9635]
Доля отклонений нулевой гипотезы Cuped_wrong: 0.9045; доверительный интервал: [0.8986, 0.9101]

Средняя длина доверительного интервала: 143.3883
Средняя длина доверительного интервала cuped: 108.3987
Средняя длина доверительного интервала Cuped_wrong: 108.3987

Средний эффект: 102.309
Средний эффект Cuped: 102.3959
Средний эффект Cuped_wrong: 102.309

Средняя дисперсия: 2596113.6468
Средняя дисперсия Cuped: 1483641.3297
Средняя дисперсия Cuped_wrong: 1483641.3297

Корреляция: 0.6543
